## Step Configuration

In [0]:
%run ./includes/configuration_jsonmoive

Out[275]: DataFrame[]

## Raw to Bronze Pipeline

In [0]:
#Step1 Read and transform raw data
rawDF = read_batch_raw(rawPath)
transformedRawDF = transform_raw(rawDF)

In [0]:
display(transformedRawDF)

Movies,Ingesttime,Status,p_Ingestdate
"{""Id"":1,""Title"":""Inception"",""Overview"":""Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \""inception\"", the implantation of another person's idea into a target's subconscious."",""Tagline"":""Your mind is the scene of the crime."",""Budget"":1.6E8,""Revenue"":8.25532764E8,""ImdbUrl"":""https://www.imdb.com/title/tt1375666"",""TmdbUrl"":""https://www.themoviedb.org/movie/27205"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//s3TBrRGB1iav7gFOCNx3H31MoES.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2010-07-15T00:00:00"",""RunTime"":148,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":6,""name"":""Action""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-01T14:54:45.972+0000,New,2022-09-01
"{""Id"":2,""Title"":""Interstellar"",""Overview"":""The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage."",""Tagline"":""Mankind was born on Earth. It was never meant to die here."",""Budget"":1.65E8,""Revenue"":6.75120017E8,""ImdbUrl"":""https://www.imdb.com/title/tt0816692"",""TmdbUrl"":""https://www.themoviedb.org/movie/157336"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//gEU2QniE6E77NI6lCU6MxlNBvIx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//xJHokMbljvjADYdit5fK5VQsXEG.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2014-11-05T00:00:00"",""RunTime"":169,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":4,""name"":""Drama""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-01T14:54:45.972+0000,New,2022-09-01
"{""Id"":3,""Title"":""The Dark Knight"",""Overview"":""Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker."",""Tagline"":""Why So Serious?"",""Budget"":1.85E8,""Revenue"":1.004558444E9,""ImdbUrl"":""https://www.imdb.com/title/tt0468569"",""TmdbUrl"":""https://www.themoviedb.org/movie/155"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//qJ2tW6WMUDux911r6m7haRef0WH.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hkBaDkMWbLaf8B1lsWsKX7Ew3Xq.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2008-07-16T00:00:00"",""RunTime"":152,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":6,""name"":""Action""},{""id"":10,""name"":""Thriller""},{""id"":11,""name"":""Crime""}]}",2022-09-01T14:54:45.972+0000,New,2022-09-01
"{""Id"":4,""Title"":""Deadpool"",""Overview"":""Deadpool tells the origin story of former Special Forces operative turned mercenary Wade Wilson, who after being subjected to a rogue experiment that leaves him with accelerated healing powers, adopts the alter ego Deadpool. Armed with his new abilities and a dark, twisted sense of humor, Deadpool hunts down the man who nearly destroyed his life."",""Tagline"":""Witness the beginning of a happy ending"",""Budget"":5.8E7,""Revenue"":7.831E8,""ImdbUrl"":""https://www.imdb.com/title/tt1431045"",""TmdbUrl"":""https://www.themoviedb.org/movie/293660"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//yGSxMiF0

In [0]:
#Step2 Write betch to a bronze table
rawToBronzeWriter = batch_writer(
    dataframe=transformedRawDF, partition_column="p_Ingestdate"
)

rawToBronzeWriter.save(bronzePath)

#Step3 Register the Bronze Table in the Metastore
spark.sql(
    """
DROP TABLE IF EXISTS movies_df_bronze
"""
)

spark.sql(
    f"""
CREATE TABLE movies_df_bronze
USING DELTA
LOCATION "{bronzePath}"
"""
)

Out[118]: DataFrame[]

## Bronze to Silver Pipeline

In [0]:
#Widgets Getting ingest date
dbutils.widgets.text('Ingest_Date','yyyy-mm-dd')
ingest_date = dbutils.widgets.get('Ingest_Date')

In [0]:
print(ingest_date)

2022-09-01

In [0]:
#Step1 Read and transform raw data
bronzeDF = read_batch_bronze(spark, ingest_date)
transformedBronzeDF = transform_bronze(bronzeDF)

In [0]:
display(bronzeDF)

Movies,Ingesttime,Status,p_Ingestdate
"{""Id"":1,""Title"":""Inception"",""Overview"":""Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \""inception\"", the implantation of another person's idea into a target's subconscious."",""Tagline"":""Your mind is the scene of the crime."",""Budget"":1.6E8,""Revenue"":8.25532764E8,""ImdbUrl"":""https://www.imdb.com/title/tt1375666"",""TmdbUrl"":""https://www.themoviedb.org/movie/27205"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//s3TBrRGB1iav7gFOCNx3H31MoES.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2010-07-15T00:00:00"",""RunTime"":148,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":6,""name"":""Action""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-01T15:04:39.659+0000,New,2022-09-01
"{""Id"":2,""Title"":""Interstellar"",""Overview"":""The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage."",""Tagline"":""Mankind was born on Earth. It was never meant to die here."",""Budget"":1.65E8,""Revenue"":6.75120017E8,""ImdbUrl"":""https://www.imdb.com/title/tt0816692"",""TmdbUrl"":""https://www.themoviedb.org/movie/157336"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//gEU2QniE6E77NI6lCU6MxlNBvIx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//xJHokMbljvjADYdit5fK5VQsXEG.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2014-11-05T00:00:00"",""RunTime"":169,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":4,""name"":""Drama""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-01T15:04:39.659+0000,New,2022-09-01
"{""Id"":3,""Title"":""The Dark Knight"",""Overview"":""Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker."",""Tagline"":""Why So Serious?"",""Budget"":1.85E8,""Revenue"":1.004558444E9,""ImdbUrl"":""https://www.imdb.com/title/tt0468569"",""TmdbUrl"":""https://www.themoviedb.org/movie/155"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//qJ2tW6WMUDux911r6m7haRef0WH.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hkBaDkMWbLaf8B1lsWsKX7Ew3Xq.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2008-07-16T00:00:00"",""RunTime"":152,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":6,""name"":""Action""},{""id"":10,""name"":""Thriller""},{""id"":11,""name"":""Crime""}]}",2022-09-01T15:04:39.659+0000,New,2022-09-01
"{""Id"":4,""Title"":""Deadpool"",""Overview"":""Deadpool tells the origin story of former Special Forces operative turned mercenary Wade Wilson, who after being subjected to a rogue experiment that leaves him with accelerated healing powers, adopts the alter ego Deadpool. Armed with his new abilities and a dark, twisted sense of humor, Deadpool hunts down the man who nearly destroyed his life."",""Tagline"":""Witness the beginning of a happy ending"",""Budget"":5.8E7,""Revenue"":7.831E8,""ImdbUrl"":""https://www.imdb.com/title/tt1431045"",""TmdbUrl"":""https://www.themoviedb.org/movie/293660"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//yGSxMiF0

In [0]:
#Step2 Generate dataframe
#clean and quarantine Movies
(silverCleanDF, silverQuarantineDF) = generate_clean_and_quarantine_dataframes(transformedBronzeDF)

#Genres
genre_df_silver = generate_genre_dataframes(transformedBronzeDF)

#Junction MovieGenres
junction_moviegenre_df_silver = generate_Juntion_MovieGenre_dataframes(transformedBronzeDF)

#OriginalLanguage
OriginalLanguage_df_silver = generate_originalLanguage_dataframes(transformedBronzeDF)


In [0]:
#Step3 WRITE Clean Batch to Silver Tables And Register
#Movies
bronzeToSilverWriter = batch_writer(
    dataframe=silverCleanDF, partition_column="p_ReleaseYear", exclude_columns=["Movies"]
)
bronzeToSilverWriter.save(silverPath)

#Genres
bronzeToSilverGenreWriter = batch_writer_lookuptable(dataframe = genre_df_silver)
bronzeToSilverGenreWriter.save(silverGenrePath)

#Junction MovieGenres
bronzeToSilverJuncMovieGenreWriter = batch_writer_lookuptable(dataframe = junction_moviegenre_df_silver)
bronzeToSilverJuncMovieGenreWriter.save(silverJuncMovieGenrePath)

#OriginalLanguage
bronzeToSilverOriginalLanguageWriter = batch_writer_lookuptable(dataframe = OriginalLanguage_df_silver)
bronzeToSilverOriginalLanguageWriter.save(silverOriginalLanguagePath)

In [0]:
#Register
#Movies
spark.sql(
    """
DROP TABLE IF EXISTS movies_df_silver
"""
)

spark.sql(
    f"""
CREATE TABLE movies_df_silver
USING DELTA
LOCATION "{silverPath}"
"""
)

#Genres
spark.sql(
    """
DROP TABLE IF EXISTS genres_df_silver
"""
)

spark.sql(
    f"""
CREATE TABLE genres_df_silver
USING DELTA
LOCATION "{silverGenrePath}"
"""
)

#Junction MovieGenres
spark.sql(
    """
DROP TABLE IF EXISTS movies_genres_df_silver
"""
)

spark.sql(
    f"""
CREATE TABLE movies_genres_df_silver
USING DELTA
LOCATION "{silverJuncMovieGenrePath}"
"""
)

#OriginalLanguage
spark.sql(
    """
DROP TABLE IF EXISTS OriLanguage_df_silver
"""
)

spark.sql(
    f"""
CREATE TABLE OriLanguage_df_silver
USING DELTA
LOCATION "{silverOriginalLanguagePath}"
"""
)


Out[295]: DataFrame[]

In [0]:
#Step4 Update Bronze table to Reflect the Loads
update_bronze_table_status(spark, bronzePath, silverCleanDF, "loaded")
update_bronze_table_status(spark, bronzePath, silverQuarantineDF, "quarantined")

Out[296]: True

## Handle Quarantined Records

In [0]:
#Step1 Load AND Transform Quarantined Records from the Bronze Table
bronzeQuarantinedDF = spark.read.table("movies_df_bronze").filter("Status = 'quarantined'")
bronzeQuarTransDF = transform_bronze(bronzeQuarantinedDF)

#Step2 Repair
repairDF = bronzeQuarTransDF.withColumn('Runtime', abs(col('Runtime'))).drop('genres')

In [0]:
#Step3 Batch Write the Repaired Dataframe
bronzeToSilverQuarWriter = batch_writer(
    dataframe=repairDF, partition_column="p_ReleaseYear", exclude_columns=["Movies"]
)

bronzeToSilverQuarWriter.save(silverPath)


In [0]:
#Step4 Update Status
update_bronze_table_status(spark, bronzePath, repairDF, "loaded")

Out[299]: True